In [68]:
import nltk
import 
import pandas as pd
from nltk.tokenize import RegexpTokenizer

#obama = pd.read_excel('C:/Users/utsav/OneDrive/UIC/Fall_2023/CS_583/Project/obama.xlsx')
#obama.head(3)

In [69]:
data = pd.ExcelFile('C:/Users/utsav/OneDrive/UIC/Fall_2023/CS_583/Project/training-Obama-Romney-tweets.xlsx')
obama = pd.read_excel(data, 'Obama')
obama = obama[1:]
obama = obama.drop(['Unnamed: 0', 'date', 'time', 'Unnamed: 5'], axis=1)
obama = obama.rename(columns={'Unnamed: 4': 'class', 'Anootated tweet': 'tweet'})
obama['tweet'] = obama['tweet'].astype(str).str.lower()
#romney = pd.read_excel(data, 'Romney')
#romney = romney[1:]
obama.head(3)

,tweet,class
1,"kirkpatrick, who wore a baseball cap embroider...",0
2,question: if <e>romney</e> and <e>obama</e> ha...,2
3,#<e>obama</e> debates that cracker ass cracker...,1


In [70]:
regexp = RegexpTokenizer('\w+')
obama['tweet_token'] = obama['tweet'].apply(regexp.tokenize)
obama.head(3)

,tweet,class,tweet_token
1,"kirkpatrick, who wore a baseball cap embroider...",0,"[kirkpatrick, who, wore, a, baseball, cap, emb..."
2,question: if <e>romney</e> and <e>obama</e> ha...,2,"[question, if, e, romney, e, and, e, obama, e,..."
3,#<e>obama</e> debates that cracker ass cracker...,1,"[e, obama, e, debates, that, cracker, ass, cra..."


In [71]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utsav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
from nltk.corpus import stopwords

In [73]:
obama['tweet_token'] = obama['tweet_token'].apply(lambda x: [item for item in x if item not in stopwords.words('english')])
obama.head(3)

,tweet,class,tweet_token
1,"kirkpatrick, who wore a baseball cap embroider...",0,"[kirkpatrick, wore, baseball, cap, embroidered..."
2,question: if <e>romney</e> and <e>obama</e> ha...,2,"[question, e, romney, e, e, obama, e, child, p..."
3,#<e>obama</e> debates that cracker ass cracker...,1,"[e, obama, e, debates, cracker, ass, cracker, ..."


In [74]:
obama['tweet_string'] = obama['tweet_token'].apply(lambda x:' '.join([item for item in x if len(item)>2]))
obama.head(5)

,tweet,class,tweet_token,tweet_string
1,"kirkpatrick, who wore a baseball cap embroider...",0,"[kirkpatrick, wore, baseball, cap, embroidered...",kirkpatrick wore baseball cap embroidered bara...
2,question: if <e>romney</e> and <e>obama</e> ha...,2,"[question, e, romney, e, e, obama, e, child, p...",question romney obama child punching contest m...
3,#<e>obama</e> debates that cracker ass cracker...,1,"[e, obama, e, debates, cracker, ass, cracker, ...",obama debates cracker ass cracker tonight tune...
4,rt @davewiner slate: blame <e>obama</e> for fo...,2,"[rt, davewiner, slate, blame, e, obama, e, fou...",davewiner slate blame obama four deaths libya ...
5,@hollivan @hereistheanswer youre missing the ...,0,"[hollivan, hereistheanswer, youre, missing, po...",hollivan hereistheanswer youre missing point a...


In [75]:
all_obama_words = ' '.join([text for text in obama['tweet_string']])
tokenized_obama = nltk.tokenize.word_tokenize(all_obama_words)
from nltk.probability import FreqDist
fdist = FreqDist(tokenized_obama)
fdist

FreqDist({'obama': 7632, 'http': 2150, 'romney': 2002, 'debate': 1111, 'president': 753, 'barack': 412, 'vote': 403, 'mitt': 378, 'like': 359, 'via': 317, ...})

In [76]:
obama['tweet_string_fdist'] = obama['tweet_token'].apply(lambda x: ' '.join([item for item in x if fdist[item] >= 1 ]))
obama.head(5)

,tweet,class,tweet_token,tweet_string,tweet_string_fdist
1,"kirkpatrick, who wore a baseball cap embroider...",0,"[kirkpatrick, wore, baseball, cap, embroidered...",kirkpatrick wore baseball cap embroidered bara...,kirkpatrick wore baseball cap embroidered bara...
2,question: if <e>romney</e> and <e>obama</e> ha...,2,"[question, e, romney, e, e, obama, e, child, p...",question romney obama child punching contest m...,question romney obama child punching contest m...
3,#<e>obama</e> debates that cracker ass cracker...,1,"[e, obama, e, debates, cracker, ass, cracker, ...",obama debates cracker ass cracker tonight tune...,obama debates cracker ass cracker tonight tune...
4,rt @davewiner slate: blame <e>obama</e> for fo...,2,"[rt, davewiner, slate, blame, e, obama, e, fou...",davewiner slate blame obama four deaths libya ...,davewiner slate blame obama four deaths libya ...
5,@hollivan @hereistheanswer youre missing the ...,0,"[hollivan, hereistheanswer, youre, missing, po...",hollivan hereistheanswer youre missing point a...,hollivan hereistheanswer youre missing point a...


In [77]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utsav\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\utsav\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [78]:
wordnet_lem = WordNetLemmatizer()
obama['tweet_string_lem'] = obama['tweet_string_fdist'].apply(wordnet_lem.lemmatize)

In [79]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\utsav\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [82]:
obama['polarity'] = obama['tweet_string_lem'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
obama.head(5)

,tweet,class,tweet_token,tweet_string,tweet_string_fdist,tweet_string_lem,polarity
1,"kirkpatrick, who wore a baseball cap embroider...",0,"[kirkpatrick, wore, baseball, cap, embroidered...",kirkpatrick wore baseball cap embroidered bara...,kirkpatrick wore baseball cap embroidered bara...,kirkpatrick wore baseball cap embroidered bara...,0.0000
2,question: if <e>romney</e> and <e>obama</e> ha...,2,"[question, e, romney, e, e, obama, e, child, p...",question romney obama child punching contest m...,question romney obama child punching contest m...,question romney obama child punching contest m...,0.0000
3,#<e>obama</e> debates that cracker ass cracker...,1,"[e, obama, e, debates, cracker, ass, cracker, ...",obama debates cracker ass cracker tonight tune...,obama debates cracker ass cracker tonight tune...,obama debates cracker ass cracker tonight tune...,-0.5423
4,rt @davewiner slate: blame <e>obama</e> for fo...,2,"[rt, davewiner, slate, blame, e, obama, e, fou...",davewiner slate blame obama four deaths libya ...,davewiner slate blame obama four deaths libya ...,davewiner slate blame obama four deaths libya ...,-0.5859
5,@hollivan @hereistheanswer youre missing the ...,0,"[hollivan, hereistheanswer, youre, missing, po...",hollivan hereistheanswer youre missing point a...,hollivan hereistheanswer youre missing point a...,hollivan hereistheanswer youre missing point a...,-0.5897


In [83]:
obama['predicted'] = obama['polarity'].apply(lambda x: 1 if x > 0 else 0 if x==0 else -1)
obama.head(5)

,tweet,class,tweet_token,tweet_string,tweet_string_fdist,tweet_string_lem,polarity,predicted
1,"kirkpatrick, who wore a baseball cap embroider...",0,"[kirkpatrick, wore, baseball, cap, embroidered...",kirkpatrick wore baseball cap embroidered bara...,kirkpatrick wore baseball cap embroidered bara...,kirkpatrick wore baseball cap embroidered bara...,0.0000,0
2,question: if <e>romney</e> and <e>obama</e> ha...,2,"[question, e, romney, e, e, obama, e, child, p...",question romney obama child punching contest m...,question romney obama child punching contest m...,question romney obama child punching contest m...,0.0000,0
3,#<e>obama</e> debates that cracker ass cracker...,1,"[e, obama, e, debates, cracker, ass, cracker, ...",obama debates cracker ass cracker tonight tune...,obama debates cracker ass cracker tonight tune...,obama debates cracker ass cracker tonight tune...,-0.5423,-1
4,rt @davewiner slate: blame <e>obama</e> for fo...,2,"[rt, davewiner, slate, blame, e, obama, e, fou...",davewiner slate blame obama four deaths libya ...,davewiner slate blame obama four deaths libya ...,davewiner slate blame obama four deaths libya ...,-0.5859,-1
5,@hollivan @hereistheanswer youre missing the ...,0,"[hollivan, hereistheanswer, youre, missing, po...",hollivan hereistheanswer youre missing point a...,hollivan hereistheanswer youre missing point a...,hollivan hereistheanswer youre missing point a...,-0.5897,-1
